In [1]:
%cd ..

C:\Users\Ihechi Festus\Documents\ML\Practice\recommendation


C:\Users\Ihechi Festus\Documents\ML\ml_env\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import re
import numpy as np
import pandas as pd
from recommendation_system import RecommenderSystem

In [3]:
pd.options.mode.copy_on_write = True

### Get Files

In [4]:
ratings = pd.read_csv(os.path.join("example", "dataset", "ratings.csv"), \
                      usecols=["userId", "movieId", "rating"]).rename(columns={"movieId": "itemId"})
items = pd.read_csv(os.path.join("example", "dataset", "movies.csv")).rename( \
                      columns={"movieId": "itemId"})
directors = pd.read_csv(os.path.join("example", "dataset", "directors.csv"), \
                      usecols=["movieId", "director"]).rename(columns={"movieId": "itemId"})

### Cleaning and Preparing Data

In [5]:
missing_years = {
        'Babylon 5': '1995',
        'Ready Player One': '2018',
        'Hyena Road': '2015',
        'The Adventures of Sherlock Holmes and Doctor Watson': '1980',
        'Nocturnal Animals': '2016',
        'Paterson': '2016',
        'Moonlight': '2016',
        'The OA': '2016',
        'Cosmos': '2019',
        'Maria Bamford: Old Baby': '2017',
        'Death Note: Desu nôto (2006–2007)': '2006',
        'Generation Iron 2': '2017',
        'Black Mirror': '2011'
    }

def split_title(title):
    """
    Given title, split into Pandas Series of [title, year]
    eg -> title: Toy Story (1995)
    Returns pd.Series["Toy Story", 1995]

    if year in given title not valid, check for release year \
    in missing_years dict, if not found, get user to input release year
    """
    match = re.search(r"\((\d{4})\)", title)
    if match:
        movie_title = title.replace(match.group(0), "").strip()
        year = match.group(1).strip()
    else:
        movie_title = title.strip()
        year = missing_years.get(movie_title, None)
        if not year:
            year = input(f"Enter movie year for {movie_title}: ")
            missing_years[movie_title] = year
    return pd.Series([movie_title, int(year)])

In [6]:
items[["title", "year"]] = items["title"].apply(split_title)
items = items.merge(directors, on="itemId")[["itemId", "title", "year", "director", "genres"]].dropna()

# ratings is 5-scale, multiply by 2 to make it 10-scale rating(You can use 5-scale, I prefer 10-scale ratings)
ratings["rating"] = ratings["rating"].mul(2)

ratings df has columns \[userId, itemId, rating]. \
items df has columns \[itemsId].extend(\[title, director]) - \[title, director] are given as item identifier. This is cause two items may have the same title, and a director may have multiple items. a combination of both usually refer to a single item

### Instantiate RecommenderSystem Class

In [19]:
file_path = "recommender_system_ratings.pkl"
recommender_system = RecommenderSystem(ratings, items, file_path, "dummy_id", item_min=30, user_min=50, \
                                       item_identifier=["title", "director"], rating_mid_point=6, max_rating=10)

#### Clean and Train Model on Dataset

In [9]:
recommender_system.clean_up_datasets()
recommender_system.train_models_and_pivot_df()

Train success


#### Get Recommendations

In [10]:
max_recommendations = 50
# Each function call generates somewhat different recommendations
recommender_system.generate_user_recommendations(max_recommendations)

[['avengers, the', 'joss whedon'],
 ['matrix, the', 'lana wachowski'],
 ['reservoir dogs', 'quentin tarantino'],
 ['iron man', 'jon favreau'],
 ['star wars: episode v - the empire strikes back', 'irvin kershner'],
 ['star wars: episode vi - return of the jedi', 'richard marquand'],
 ['iron man 2', 'jon favreau'],
 ['star wars: episode iv - a new hope', 'george lucas'],
 ['pirates of the caribbean: the curse of the black pearl', 'gore verbinski'],
 ['interstellar', 'christopher nolan'],
 ['avatar', 'james cameron'],
 ['indiana jones and the last crusade', 'steven spielberg'],
 ['django unchained', 'quentin tarantino'],
 ['bourne identity, the', 'doug liman'],
 ['x-men: first class', 'matthew vaughn'],
 ['forrest gump', 'robert zemeckis'],
 ['guardians of the galaxy', 'james gunn'],
 ['departed, the', 'martin scorsese'],
 ['harry potter and the goblet of fire', 'mike newell'],
 ['lord of the rings: the fellowship of the ring, the', 'peter jackson'],
 ['inglourious basterds', 'quentin tar

#### Update Ratings

In [11]:
rating = 10 # what you rate the item
item = ['dark knight rises, the', 'christopher nolan'] # follows item_identifier pattern
recommender_system.update_users_ratings(item, rating)

Your ratings dataset has been updated!


After updating ratings, you can rerun the above cell block(`recommender_system.generate_user_recommendations`) to get newer recommendations based on your updated ratings


#### View Your Ratings
should you want to

In [20]:
recommender_system.retrieve_user_ratings()#[["title", "director"]]#.to_numpy()[:, 1]

,itemId,rating,userId,title,director
